In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [6]:
random_state = 59
np.random.seed(random_state)
torch.manual_seed(random_state)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_state)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
dataset_path = 'Auto_MPG_data.csv'
dataset = pd.read_csv(dataset_path)

In [11]:
X = dataset.drop(columns='MPG').values
y = dataset['MPG'].values

In [14]:
val_size = 0.2
test_size = 0.125

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = val_size, random_state=random_state)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=test_size, random_state=random_state)

In [15]:
normalizer = StandardScaler()
X_train = normalizer.fit_transform(X_train)
X_val = normalizer.transform(X_val)
X_test = normalizer.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [16]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [17]:
batch_size = 32
train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)
test_dataset = CustomDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [18]:
class MLP(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims):
        super().__init__()
        self.linear1 = nn.Linear(input_dims, hidden_dims)
        self.linear2 = nn.Linear(hidden_dims, hidden_dims)
        self.output = nn.Linear(hidden_dims, output_dims)
    
    def forward(self, x):
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x)
        out = self.output(x)

        return out.squeeze(1)

In [19]:
input_dims = X_train.shape[1]
output_dims = 1
hidden_dims = 64

model = MLP(input_dims, hidden_dims, output_dims).to(device)

In [20]:
lr = 1e-2
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [21]:
def r_squared(y_true, y_pred):
    y_true = torch.Tensor(y_true).to(device)
    y_pred = torch.Tensor(y_pred).to(device)
    mean_true = torch.mean(y_true)
    ss_tot = torch.sum((y_true - mean_true) ** 2)
    ss_res = torch.sum((y_true - y_pred) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2

In [26]:
epochs = 1000
train_losses = []
val_losses = []
train_r2 = []
val_r2 = []

for epoch in range(epochs):
    train_loss = 0.0
    train_target = []
    val_target = []
    train_predict = []
    val_predict = []
    model.train()

    for X_samples, y_samples in train_loader:
        X_samples = X_samples.to(device)
        y_samples = y_samples.to(device)
        optimizer.zero_grad()
        outputs = model(X_samples)
        train_predict += outputs.tolist()
        train_target += y_samples.tolist()
        loss = criterion(outputs, y_samples)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    train_r2.append(r_squared(train_target, train_predict))
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_samples, y_samples in val_loader:
            X_samples = X_samples.to(device)
            y_samples = y_samples.to(device)
            outputs = model(X_samples)
            val_predict += outputs.tolist()
            val_target += y_samples.tolist()
            loss = criterion(outputs, y_samples)
            val_loss += loss.item()

    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    val_r2.append(r_squared(val_target, val_predict))

    print(f'\nEPOCH {epoch + 1}:\tTraining loss: {train_loss :.3f}\tValidation loss: {val_loss :.3f}')



EPOCH 1:	Training loss: 90.361	Validation loss: 20.327

EPOCH 2:	Training loss: 17.722	Validation loss: 14.955

EPOCH 3:	Training loss: 20.481	Validation loss: 10.068

EPOCH 4:	Training loss: 38.832	Validation loss: 38.442

EPOCH 5:	Training loss: 19.056	Validation loss: 36.808

EPOCH 6:	Training loss: 23.355	Validation loss: 43.035

EPOCH 7:	Training loss: 29.892	Validation loss: 22.099

EPOCH 8:	Training loss: 10.058	Validation loss: 5.395

EPOCH 9:	Training loss: 15.174	Validation loss: 15.289

EPOCH 10:	Training loss: 12.548	Validation loss: 14.489

EPOCH 11:	Training loss: 15.171	Validation loss: 8.579

EPOCH 12:	Training loss: 11.173	Validation loss: 20.875

EPOCH 13:	Training loss: 12.929	Validation loss: 14.357

EPOCH 14:	Training loss: 15.647	Validation loss: 12.957

EPOCH 15:	Training loss: 15.186	Validation loss: 5.237

EPOCH 16:	Training loss: 7.288	Validation loss: 5.030

EPOCH 17:	Training loss: 8.615	Validation loss: 4.650

EPOCH 18:	Training loss: 6.197	Validation loss

In [29]:
model.eval()
with torch.no_grad():
    y_hat = model(X_test)
    test_set_r2 = r_squared(y_hat, y_test)
    print(f'\nTest R^2: {test_set_r2}')


Test R^2: 0.8523630499839783
